In [1]:
import pandas as pd
import glob
import os

## Porównanie metod

In [2]:
def read_all_files(path):
    all_files = glob.glob(os.path.join('.', path + "/*.txt"))
    li = []

    for filename in all_files:
        print(filename)
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)
    return li

In [6]:
li = read_all_files("data/2blue-vs-1red")
frame = pd.concat(li, axis=0, ignore_index=True)
frame

./data/2blue-vs-1red/ModelOutput.2019.lis.17.20_36_20.txt
./data/2blue-vs-1red/ModelOutput.2019.lis.17.20_36_12.txt
./data/2blue-vs-1red/ModelOutput.2019.lis.17.20_36_25.txt
./data/2blue-vs-1red/ModelOutput.2019.lis.17.20_35_48.txt
./data/2blue-vs-1red/ModelOutput.2019.lis.17.20_36_04.txt


,tick,run,remainingTanksBlue,remainingTanksRed,remainingRidersBlue,remainingRidersRed,remainingSoldiersBlue,remainingSoldiersRed
0,1.0,1,10.0,10.0,10.0,10.0,10.0,10.0
1,2.0,1,10.0,10.0,10.0,10.0,10.0,10.0
2,3.0,1,10.0,10.0,10.0,10.0,10.0,10.0
3,4.0,1,10.0,10.0,10.0,10.0,10.0,10.0
4,5.0,1,10.0,10.0,9.0,10.0,10.0,10.0
...,...,...,...,...,...,...,...,...
1065,176.0,1,2.0,1.0,1.0,0.0,0.0,0.0
1066,177.0,1,2.0,1.0,1.0,0.0,0.0,0.0
1067,178.0,1,2.0,1.0,1.0,0.0,0.0,0.0
1068,179.0,1,2.0,1.0,1.0,0.0,0.0,0.0


## Porównanie liczności agentów różnego typu

In [4]:
def read_one_file(path):
    return pd.read_csv(os.path.join('.', path), index_col=None, header=0)

In [5]:
mapping = read_one_file("data/0/ModelOutput.2019.lis.17.20_08_16.batch_param_map.txt")
mapping

data = read_one_file("data/0/ModelOutput.2019.lis.17.20_08_16.txt")
data

,tick,run,remainingTanksBlue,remainingTanksRed,remainingRidersBlue,remainingRidersRed,remainingSoldiersBlue,remainingSoldiersRed
0,1.0,1,10.0,10.0,10.0,10.0,10.0,10.0
1,2.0,1,10.0,10.0,10.0,10.0,10.0,10.0
2,3.0,1,10.0,10.0,9.0,10.0,10.0,10.0
3,4.0,1,10.0,9.0,9.0,10.0,10.0,10.0
4,5.0,1,10.0,9.0,8.0,9.0,10.0,10.0
...,...,...,...,...,...,...,...,...
477136,402.0,729,0.0,0.0,1.0,1.0,0.0,0.0
477137,403.0,729,0.0,0.0,1.0,1.0,0.0,0.0
477138,404.0,729,0.0,0.0,1.0,1.0,0.0,0.0
477139,405.0,729,0.0,0.0,1.0,1.0,0.0,0.0
